In [1]:
## Rolar un usuario. Genera los datos de train y Val
import cv2
import os
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from os import listdir
from keras.models import load_model
from numpy import savez_compressed
import numpy as np
import random

Using TensorFlow backend.


In [2]:
def tomarfoto(user):
    print('Presiona la tecla g para capturar la foto')
    print('Presiona la tecla q para salir')
    cap = cv2.VideoCapture(0)
    fotouser =[]
    while True:
        ret,frame = cap.read()
        if not ret:
            print('Fallo en lectura de la camara')
            break
        key = cv2.waitKey(1)
        if key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        cv2.imshow('Camera', frame)
        if key == ord('g'):
            fotouser = frame     #Guarda foto
            print('foto capturada exitosamente')
    return fotouser    # La funcion retorna la foto

In [3]:
def Getear(foto, user):
    copia = foto
    earClassif = cv2.CascadeClassifier('cascade.xml')
    gray = cv2.cvtColor(copia, cv2.COLOR_BGR2GRAY)
    ear = earClassif.detectMultiScale(gray, scaleFactor=1.07, minNeighbors=10)  ##minNeig se cambia si no reconoce oreja
    out = []
    for (x,y,w,h) in ear:
        out = foto[y-5:y+h+10,x-10:x+w+10]
    plt.title("foto ear")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    plt.imshow(out)
    plt.show()
    return out

In [4]:
def Aumentardatos(ear):
    
    #create image data augmentation generator

    datagenb = ImageDataGenerator(brightness_range=[0.6,1.4])
    datagenr = ImageDataGenerator(rotation_range= (16))
    
    #data = img_to_array(ear)
    samples = expand_dims(ear, 0)
    itb = datagenb.flow(samples, batch_size=1)
    itr = datagenr.flow(samples, batch_size=1)
    
    fotos = list()
    j = 0
    for i in range(30):   #generara tantas como quiera cada iteracion saca una modificacion de la imagen original
        
        z = round(random.uniform(0.94,0.86),3)
        datagenz = ImageDataGenerator(zoom_range=[z,z])        ## Zoom acerca
        a = round(random.uniform(1.02,1.11),3)
        datagena = ImageDataGenerator(zoom_range=[a,a])       ## aleja

        itz = datagenz.flow(samples, batch_size=1)
        ita = datagena.flow(samples, batch_size=1)
        
        # generate batch of images
        batchz = itz.next()
        batcha = ita.next()
        batchr = itr.next()
        batchb = itb.next()
        
        fotos.append(batchz[0].astype('uint8'))
        fotos.append(batcha[0].astype('uint8'))
        fotos.append(batchr[0].astype('uint8'))
        fotos.append(batchb[0].astype('uint8'))
    print('Datos aumentados guardados')
    return fotos

In [6]:
####Embedings#######
# get the ear embedding for one ear
def get_embedding(model, ear_pixels):
    required_size = (160,160)
    ear = cv2.resize(ear_pixels, required_size)   #(160,160)   #oreja lista
    # scale pixel values
    ear = ear.astype('float32')
    # standardize pixel values across channels (global)  #esto es requisito para el modelo Facenet
    mean, std = ear.mean(), ear.std()
    ear = (ear - mean) / std
    # transform ear into one sample
    samples = expand_dims(ear, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [7]:
def AllEmbedings(data, user, code):
    # code es los ultimos 3 digitos de la cedula del usuario
    model  = load_model('facenet_keras.h5')
    Emb    = list()
    Label  = list()
    DataBase = 'DataBaseTemp/'
    
    for ear in data:
        emb = get_embedding(model, ear)
        Emb.append(emb)
        Label.append(user)

    savez_compressed(DataBase + code + '.npz',Emb,Label)    ##embedings solo del usuario
    print('El usuario ' + user + ' fue registrado exitosamente')

In [8]:
def Rolar (user, code):
    print('Ingrese nombre de usuario: ')
    user = input()
    print('Ingrese codigo (ultimos 3 digitos de la cedula) : ')
    code = input()
    foto   = tomarfoto(user)
    ear    = Getear(foto, user)
    print('¿Continuar? Si : y | No : n')
    conti = input()
    if conti == 'y':
        orejas = Aumentardatos(ear)
        AllEmbedings(orejas, user, code)